In [2]:
import json
import pandas
import math
import matplotlib.pyplot as plt
import itertools
import hashlib
from string import Template
import requests
import shutil
import os.path
import time
session = requests.Session()

%pylab 

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [8]:
with open('autotradevans.json') as fp, open('kijjivans.json') as fp2, open('vans.json') as fp3, open('repos.json') as fp4:
    autotrade_vans = [json.loads(line) for line in fp]
    for van in autotrade_vans: van['crawler'] = 'autotrade'
    kijji_vans = [json.loads(line) for line in fp2]
    for van in kijji_vans: van['crawler'] = 'kijji'
    craigslist_vans = [json.loads(line) for line in fp3]
    for van in craigslist_vans: van['crawler'] = 'craigslist'
    repos_vans = [json.loads(line) for line in fp4]
    vans = autotrade_vans + kijji_vans + craigslist_vans + repos_vans
    
    print (len(vans))

947


In [3]:
def preprocess_vans(vans):
    for van in vans:
        odometer = '0'
        if 'odometer' in van:
            odometer = str(van['odometer'])
            
        if 'Kilometers' in van:
            odometer = van['Kilometers']
            
        if 'Kilometres' in van:
            odometer = van['Kilometres']
        
        odometer = odometer.replace(',', '').replace('km', '').strip()

        odometer = float(odometer)

        if odometer < 1000:
            odometer = odometer * 1000
        van['odometer'] = odometer
        
        if 'Price' in van:
            van['price'] = van['Price']
        
        if 'query' not in van:
            van['query'] = "Autotrader"
            
        van['price'] = str(van['price']).replace(',', '').replace('$', '')
        van['price'] = float(van['price'])
        try:
            van['city'] = van['city'].lower()
        except:
            van['city'] = ' '.join(van['url'].split('/')[4].split('-')[:-1])
            
        
    
preprocess_vans(vans)

In [4]:
frame = pandas.DataFrame(vans)
frame['value'] = frame['price'] / frame['odometer']
frame = frame.drop_duplicates(subset=['price', 'title', 'odometer'])

In [9]:
print (len(repos_vans))

20


In [6]:
canadian_cities = ["vancouver", "abbotsford", "whistler", "kamloops", "kelowna"]
us_cities = ['bellingham', 'seattle']
localonly = (~frame['city'].isin(us_cities)) & (frame['price'] < 10000)
points = frame[localonly]
lookup = {}
next_index = 0
for index, isLocal in zip(itertools.count(), localonly):
    if isLocal:
        lookup[next_index] = index
        next_index += 1
        
def onclick(event):
    print (event.ind)
    for index in event.ind:
        point = points.xs(lookup[index])
        print (point)
        print (point['url'])

def color(query):
    if query == "dodge ram van": return "red"
    elif query == "chevy express": return "blue"
    elif query == "gmc savana": return "green"
    elif query == "cargo van": return "magenta"
    elif query == "ford transit": return "purple"
    elif query.startswith("ford"): return "black"
    else: return "yellow"
    
        
fig = plt.figure(figsize=(20, 10))
print(sum(localonly))
colors = [color(query) for query in frame[localonly]["query"]]
plot = plt.scatter(points['odometer'], points['price'], color=colors, picker=True)  
cid = fig.canvas.mpl_connect('pick_event', onclick)

257


In [45]:
pandas.set_option('display.expand_frame_repr', False)
pandas.set_option('display.max_colwidth', 300)
pandas.set_option('display.width', 1000)
pandas.set_option('display.max_rows', 1000)

points[['value', 'title', 'url', 'city', 'price', 'odometer']].sort_values('value', ascending=False)


,value,title,url,city,price,odometer
187,inf,Ford Motorhome,http://www.kijiji.ca/v-rv-motorhome/dawson-creek/ford-motorhome/1182893772,dawson,6800.0,0.0
21,inf,NaN,http://wwwa.autotrader.ca/a/GMC/Sierra+3500+Cab-Chassis/Langley/British+Columbia/5_23789946_20060830173625256/?ms=heavy_trucks&showcpo=ShowCPO&orup=22_100_43,langley,7995.0,0.0
185,inf,rv fridge freezer,http://www.kijiji.ca/v-trailer-parts-accessories/revelstoke/rv-fridge-freezer/1190564680,,400.0,0.0
484,inf,GREAT LITTLE TRAILER 1992 jAYCO LIKE NEW,http://abbotsford.craigslist.ca/rvs/5754596252.html,abbotsford,3800.0,0.0
245,inf,LIKE NEW 1992 JAYCO TENT TRILER,http://vancouver.craigslist.ca/rds/rvs/5754582504.html,vancouver,3800.0,0.0
8,8.995000,NaN,http://wwwa.autotrader.ca/a/GMC/Savana/Langley/British+Columbia/5_28203530_20060830173625256/?ms=heavy_trucks&showcpo=ShowCPO&orup=9_100_43,langley,8995.0,1000.0
22,7.995000,NaN,http://wwwa.autotrader.ca/a/Chevrolet/Express/Langley/British+Columbia/5_28793960_20060830173625256/?ms=heavy_trucks&showcpo=ShowCPO&orup=23_100_43,langley,7995.0,1000.0
29,6.995000,NaN,http://wwwa.autotrader.ca/a/Ford/F-550/Langley/British+Columbia/5_27262452_20060830173625256/?ms=heavy_trucks&showcpo=ShowCPO&orup=30_100_43,langley,6995.0,1000.0
42,1.500000,NaN,http://wwwa.autotrader.ca/a/Chevrolet/2500+Cab-Chassis/KELOWNA/British+Columbia/19_9545902_/?ms=heavy_trucks&showcpo=ShowCPO&orup=43_100_43,kelowna,1500.0,1000.0
36,0.445525,NaN,http://wwwa.autotrader.ca/a/Chevrolet/2500+Cab-Chassis/PRINCE+GEORGE/British+Columbia/19_9489803_/?ms=heavy_trucks&showcpo=ShowCPO&orup=37_100_43,george,5500.0,12345.0


In [46]:
frame[localonly][['crawler', 'city', 'url']].groupby(['crawler', 'city']).count()

url
crawler    city                        
autotrade  abbotsford                 2
           burnaby                    4
           chilliwack                 2
           dewdney                    1
           george                     1
           john                       1
           kamloops                   2
           kelowna                    3
           langley                    5
           parksville                 1
           quesnel                    1
           ridge                      3
           rock                       1
           surrey                    10
           vancouver                  4
           victoria                   1
           westminster                1
craigslist abbotsford                14
           calgary                    2
           comoxvalley                1
           edmonton                   3
           kamloops                   2
           kelowna                    5
           moseslake                  3
           nanaimo                    4
           olympic                    4
           reddeer                    1
           skagit                     2
           vancouver                119
           victoria                   5
           wenatchee                  2
           whistler                   5
           yakima                     2
kijji                                21
           burnaby new                2
           dawson                     1
           delta surrey               2
           parksville qualicum        1
           port                       2
           prince                     6
           tricities pitt             1
           vancouver downtown west    1
           victoria                   2
           williams                   1

In [47]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579 entries, 0 to 917
Data columns (total 61 columns):
Accident History Report(What's This?)    30 non-null object
Address                                  45 non-null object
Body Type                                76 non-null object
City Fuel Economy                        10 non-null object
Colour                                   40 non-null object
Cylinders                                11 non-null object
Date Listed                              45 non-null object
Doors                                    57 non-null object
Drivetrain                               74 non-null object
Engine                                   37 non-null object
Exterior Colour                          65 non-null object
For Sale By                              45 non-null object
Fuel Type                                85 non-null object
Hwy Fuel Economy                         10 non-null object
Interior Colour                          30 non-null ob

In [7]:
carasol = Template('''<div id='$name' class='carousel slide' data-ride='carousel'>
    <!-- Indicators -->
    <ol class='carousel-indicators'>
      $labels
    </ol>

    <!-- Wrapper for slides -->
    <div class='carousel-inner' role='listbox'>
      $images
    </div>

    <!-- Left and right controls -->
    <a class='left carousel-control' href='#$name' role='button' data-slide='prev'>
      <span class='glyphicon glyphicon-chevron-left' aria-hidden='true'></span>
      <span class='sr-only'>Previous</span>
    </a>
    <a class='right carousel-control' href='#$name' role='button' data-slide='next'>
      <span class='glyphicon glyphicon-chevron-right' aria-hidden='true'></span>
      <span class='sr-only'>Next</span>
    </a>
  </div>''')
label = Template('''<li data-target='#$name' data-slide-to='$count'></li>''')
image = Template('''<div class='item$active'><img src='$image'></div>''')

In [9]:
with open('aggregate.json', 'w') as fp:
  interesting = ['value', 'title', 'url', 'city', 'price', 'odometer', 'images']
  local_frame = frame[localonly].fillna("")
  records = local_frame[interesting].to_dict(orient='records')
pandas.set_option('display.max_columns', None)

In [10]:
def get_image_name(url):
    if url.endswith('/$_35.JPG'):
        return url.split('/')[-2] + ".jpg"
    return url.split('/')[-1]

In [63]:
urls = [url for record in records if type(record['images']) is not float
            for url in record['images'] ]

for url in urls:
    image_name = "images/%s" % get_image_name(url)
    if (os.path.exists(image_name)): continue
    print (url)
    response = requests.get(url, stream=True)
    try:
        with open(image_name, 'wb') as out_file:
          shutil.copyfileobj(response.raw, out_file)
        del response
        time.sleep(1)
    except:
        print ("failed: ", url)
        time.sleep(60)

http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/scQAAOSw8w1X3qmF/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/Y0YAAOSwzaJX3qmY/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/2tMAAOSw4shX3qmw/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/ddsAAOSwOyJX3qnN/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/XxwAAOSwCGVX3qn7/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/dVIAAOSwmLlX3qoT/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/6oIAAOSw4shX3qok/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/Bz4AAOSw8gVX3qoy/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/EEwAAOSwLF1X3qpE/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/HpUAAOSwLnBX3qpX/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/fdoAAOSwmfhX3VzT/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/9EwAAOSwTA9X3VzS/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/9FsAAOSwTA9X3VzT/$_35.JPG
http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/Zr4AAOSwCGVX3VzT/$_35.JPG
http://i.ebayimg.com/00/s/ODAwWDYwMA==/z/g90AAOSw8gVX3VzV/$_35

In [ ]:
for record in records:
    images = []
    labels = []
    name = hashlib.sha256(record['url'].encode('utf-8')).hexdigest()
    if type(record['images']) == float: 
        record['images'] = '<div></div>'
        continue
        
    for image_url, index in zip(record['images'], itertools.count()):
        isActive = " active" if not labels else ""
        images += [image.substitute(image = "images/%s" % get_image_name(image_url), active = isActive)]
        labels += [label.substitute(name=name, count=index)]
    images = ' '.join(images)
    labels = ' '.join(labels)
    actions = ["<a href='hide/%s'><img border='0' src='images/x.png' width='32' height='32'></a>" % name]
    actions += ["<a href='favourite/%s'><img border='0' src='images/star_gold_256.png' width='32' height='32'></a>" % name]
    actions += ["<a href='%s'><img border='0' src='images/goto.png' width='32' height='32'></a>" % record['url']]
    record['actions'] = ''.join(actions)
    record['name'] = name
    record['images'] = carasol.substitute(name=name, images=images, labels=labels)

In [ ]:
with open('carasol.json', 'w') as fp:
    json.dump(records, fp, indent = 4)

In [ ]:
with open('carasol.json') as fp, open('images/carasol.json', 'w') as fp2:
    for line in fp:
        line = line.replace('NaN', '""')
        line = line.replace('Infinity', "0")
        fp2.write(line)